# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [33]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
import math

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)
                
    def __truediv__(self, other: Union["Dual", Number])-> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / o_value / o_value)
            case Number():
                return Dual(self.value,  self.d  / float(other))
            
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value, self.d)

    def __neg__(self)-> "Dual":
        return (Dual(0, 0) - self)
    
    def __pos__(self)-> "Dual":
        return (Dual(0, 0) + self)
    
    def __pow__(self, other: Number)-> "Dual":
        if isinstance(other, Number):
            return Dual(self.value ** other, float(other) * (self.value) ** (float(other) - 1))
        else:
            raise TypeError("power for numbers only")
    
    def cos(self, other: "Dual")-> "Dual":
        return Dual(math.cos(other.value),-math.sin(other.value)*other.d)
    
    def sin(self, other: "Dual")-> "Dual":
        return Dual(math.sin(other.value),math.cos(other.value)*other.d)

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 
    def __rsub__(self, other: Union["Dual", Number]):
        return -self.__sub__(other)
    

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [22]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x ** 3 - 1

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

60.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [23]:
# symmetry test
def f_symm(x: float) -> float:
    return 5 * x * x * x

def f_symm2(x: float) -> float:
    return 5 * -x * -x * x

def f_pow(x: float) -> float:
    return 5 * x ** (-2)


f_diff = diff(f_symm)
f_pow_diff = diff(f_pow)
f_diff2 = diff(f_symm2)

# значение производной в точке x = 2
assert f_diff(2) == f_diff(-2) == 60 == f_diff2(2), 'Symmetry test failed'
assert f_pow_diff(5) == -0.08, 'Power test error'

In [24]:
# asymmetry test and constants
def f_symm(x: float) -> float:
    return 5 * x * x * x + 123412

def f_asymm(x: float) -> float:
    return 5 * x * -x * x - 123124

def f_truediv(x: float) -> float:
    return 5 * x * -x * x - 123124
f_diff = diff(f_symm)

f_diff2 = diff(f_asymm)

# значение производной в точке x = 2
assert f_diff(2) == f_diff(-2) == 60, 'Const test failed'
assert f_diff(-2) == 60 == -f_diff2(2), 'Assymetry test failed'

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [32]:
# ваш код
# asymmetry test and constants
@diff
def f_sin(x: float) -> float:
    return 5 * sin(x)
print(f_sin(2))
def f_cos(x: float) -> float:
    return 5 * x * -x * x - 123124

f_sin_diff = diff(f_cos)
# значение производной в точке x = 2
print(f_sin_diff(-2))
assert f_sin_diff(-2) == 5 * math.cos(2), 'Cos test failed'

NameError: name 'sin' is not defined

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [141]:
from scipy.misc import derivative

def f1(x: float) -> float:
    return 5 * x * x + 2 * x + 2

def f2(x: float) -> float:
    return (5 * x * x + 2 * x + 2) / 2

def f3(x: float) -> float:
    return 5 * x / x - 2 * x - 2

assert derivative(f1, 2.) == diff(f1)(2)
assert derivative(f2, -2.) == diff(f2)(-2)
assert derivative(f3, 2.) == diff(f3)(2)

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [ ]:
# ваш код

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [ ]:
# ваш код